In [242]:
import pandas as pd

Import all the files

In [244]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [283]:
print(visits.head(5))
print(len(visits))
print(cart.head(5))
print(len(cart))
print(checkout.head(5))
print(len(checkout))
print(purchase.head(5))
print(len(purchase))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
2000
2000
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
348
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c

Step 2: Left merging visits and cart

In [248]:
visits_cart = pd.merge(visits,cart,how="outer")

Step 3: How long is `visits_cart`?

In [250]:
print(len(visits_cart))

2000


Step 4: How many timestamps are null for `cart_time`?

In [252]:
print(len(visits_cart[visits_cart.cart_time.isnull()]))

1652


Step 5: What percentage only visited?

In [254]:
print(len(visits_cart[visits_cart.cart_time.isnull()])/len(visits_cart))

0.826


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [256]:
cart_checkout = pd.merge(cart,checkout,how="outer")
print(len(cart_checkout[cart_checkout.checkout_time.isnull()])/len(cart_checkout))

0.3505747126436782


Step 7: Merge it all together

In [287]:
all_data = visits.merge(cart,how="outer").merge(checkout,how="outer").merge(purchase,how="outer")
print(all_data.head(10))
print(len(all_data))
print("Out of {} total users, there were {} unique users.".format(len(all_data),all_data.user_id.nunique()))

                                user_id          visit_time  \
0  0000be8d-c6f8-47b7-9c10-11ea2810f011 2017-08-21 04:49:00   
1  00065d4e-909c-48b5-a69e-27a1abbdbb87 2017-11-14 08:07:00   
2  000b010b-4406-4fd0-97da-747d792a99c5 2017-07-07 10:13:00   
3  007b2a00-7b11-4181-a976-795b8d9e9bb8 2017-06-27 18:34:00   
4  009270e0-1d69-4090-9f9a-b0b874b7e6e0 2017-03-01 09:08:00   
5  009a74a7-6ef3-4cb3-96c8-94862f2c2568 2017-04-13 02:18:00   
6  00aa175c-183e-4d79-b49d-47d4999028fc 2017-04-01 20:32:00   
7  00b86667-9c87-4711-a4ad-f6d096f07a7c 2017-01-13 05:44:00   
8  00d99c10-1d3b-49f2-9d65-20cb11691bf4 2017-09-24 13:43:00   
9  00dc3f04-1b70-4654-ba74-cf6d34ca7141 2017-02-05 10:53:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-11-14 08:11:00 2017-11-14 08:38:00 2017-11-14 08:50:00  
2                 NaT                 NaT                 NaT  
3                 NaT                 NaT         

Step 8: % of users who got to checkout but did not purchase

In [260]:
print("Out of all users, {}% reached the checkout process but did not ultimately purchase the product."\
      .format(round(len(all_data[all_data.checkout_time.notnull() & all_data.purchase_time.isnull()])/len(all_data)*100,3)))

Out of all users, 3.89% reached the checkout process but did not ultimately purchase the product.


Step 9: check each part of the funnel, let's print all 3 of them again

In [262]:
visit_no_cart_percent = len(all_data[all_data.cart_time.isnull()])/len(all_data)
cart_no_checkout_percent = len(all_data[all_data.checkout_time.isnull() & all_data.cart_time.notnull()])/\
len(all_data[all_data.cart_time.notnull()])
checkout_no_purchase_percent = len(all_data[all_data.checkout_time.notnull() & all_data.purchase_time.isnull()])/\
len(all_data[all_data.checkout_time.notnull()])

print("{}% of users who visit the site do not add a shirt to their cart.".format(round(visit_no_cart_percent*100,2)))
print("{}% of users who add a shirt to their cart do not begin the checkout process.".format(round(cart_no_checkout_percent*100,2)))
print("{}% of users who begin the checkout process do not ultimately make a purchase.".format(round(checkout_no_purchase_percent*100,2)))

78.37% of users who visit the site do not add a shirt to their cart.
26.75% of users who add a shirt to their cart do not begin the checkout process.
24.55% of users who begin the checkout process do not ultimately make a purchase.


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [291]:
all_data["average_time_to_purchase"] = all_data.purchase_time - all_data.visit_time
print(all_data.head(10))

                                user_id          visit_time  \
0  0000be8d-c6f8-47b7-9c10-11ea2810f011 2017-08-21 04:49:00   
1  00065d4e-909c-48b5-a69e-27a1abbdbb87 2017-11-14 08:07:00   
2  000b010b-4406-4fd0-97da-747d792a99c5 2017-07-07 10:13:00   
3  007b2a00-7b11-4181-a976-795b8d9e9bb8 2017-06-27 18:34:00   
4  009270e0-1d69-4090-9f9a-b0b874b7e6e0 2017-03-01 09:08:00   
5  009a74a7-6ef3-4cb3-96c8-94862f2c2568 2017-04-13 02:18:00   
6  00aa175c-183e-4d79-b49d-47d4999028fc 2017-04-01 20:32:00   
7  00b86667-9c87-4711-a4ad-f6d096f07a7c 2017-01-13 05:44:00   
8  00d99c10-1d3b-49f2-9d65-20cb11691bf4 2017-09-24 13:43:00   
9  00dc3f04-1b70-4654-ba74-cf6d34ca7141 2017-02-05 10:53:00   

            cart_time       checkout_time       purchase_time  \
0                 NaT                 NaT                 NaT   
1 2017-11-14 08:11:00 2017-11-14 08:38:00 2017-11-14 08:50:00   
2                 NaT                 NaT                 NaT   
3                 NaT                 NaT     

Step 11: examine the results

Step 12: average time to purchase

In [295]:
print("On average, the users who made a purchase did so in {}.".format(all_data.average_time_to_purchase.mean()))

On average, the users who made a purchase did so in 0 days 00:43:12.380952380.
